# 🤖 AI, CA3, Machine Learning 📚  

* **Name** : Mohammad Sadra 🖊  
* **Last Name** :  Abbasi 📝  
* **SID** : 810101469 🆔

# IMPORT


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


# Preprocess

In [ ]:
from sklearn.preprocessing import StandardScaler

class Preprocessor:
    def __init__(self, target_col='crisis_type', numerical_strategy='mean', categorical_strategy='most_frequent'):
        self.target_col = target_col
        self.num_strategy = numerical_strategy
        self.cat_strategy = categorical_strategy
        
        self.num_imputer = None
        self.cat_imputer = None
        self.scaler = StandardScaler()
        self.label_encoders = {}
        self.target_encoder = LabelEncoder()
        
        self.num_cols = []
        self.cat_cols = []
        self.outlier_bounds = {}

    def _feature_engineering(self, df):
        df_copy = df.copy()
                    
        if 'avg_sleep_last_week' in df_copy.columns and 'sleep_hours_before_exam' in df_copy.columns:
            df_copy['sleep_stability'] = abs(df_copy['avg_sleep_last_week'] - df_copy['sleep_hours_before_exam'])
            
        return df_copy

    def fit(self, X, y=None):
        X = self._feature_engineering(X)
        
        self.num_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
        self.cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
        
        if self.target_col in self.cat_cols: self.cat_cols.remove(self.target_col)
        
        self.num_imputer = SimpleImputer(strategy=self.num_strategy)
        self.num_imputer.fit(X[self.num_cols])
        
        self.cat_imputer = SimpleImputer(strategy=self.cat_strategy)
        self.cat_imputer.fit(X[self.cat_cols])
        
        for col in self.num_cols:
            Q1 = X[col].quantile(0.01)
            Q3 = X[col].quantile(0.99)
            self.outlier_bounds[col] = (Q1, Q3)

        for col in self.cat_cols:
            le = LabelEncoder()
            le.fit(X[col].astype(str))
            self.label_encoders[col] = le
            
        if y is not None:
            self.target_encoder.fit(y)
            
        X_temp_num = self.num_imputer.transform(X[self.num_cols])
        self.scaler.fit(X_temp_num)
        
        return self

    def transform(self, X, y=None):
        X = X.copy()
        
        X = self._feature_engineering(X)
        
        X[self.num_cols] = self.num_imputer.transform(X[self.num_cols])
        X[self.cat_cols] = self.cat_imputer.transform(X[self.cat_cols])
        
        for col in self.num_cols:
            lower, upper = self.outlier_bounds[col]
            X[col] = np.clip(X[col], lower, upper)
            
        for col in self.cat_cols:
            le = self.label_encoders[col]
            X[col] = X[col].astype(str).map(lambda s: s if s in le.classes_ else le.classes_[0])
            X[col] = le.transform(X[col])
            
        X[self.num_cols] = self.scaler.transform(X[self.num_cols].values)
        
        y_transformed = None
        if y is not None:
            y_transformed = self.target_encoder.transform(y)
            
        if y is not None:
            return X, y_transformed
        else:
            return X
        
    def plot_correlation(self, X, y=None):
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        df_temp = pd.DataFrame(X, columns=self.num_cols + self.cat_cols)
        
        if y is not None:
            df_temp[self.target_col] = y
            
        plt.figure(figsize=(12, 10))
        sns.heatmap(df_temp.corr(), annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
        plt.title("Feature Correlation Matrix")
        plt.show()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('./part3_dataset/part3_dataset.csv')

X_raw = data.drop('crisis_type', axis=1)
y_raw = data['crisis_type']

X_train_raw, X_val_raw, y_train_raw, y_val_raw = train_test_split(X_raw, y_raw, test_size=0.3, random_state=42)

preprocessor = Preprocessor(target_col='crisis_type')
preprocessor.fit(X_train_raw, y_train_raw)

X_train, y_train = preprocessor.transform(X_train_raw, y_train_raw)
X_val, y_val = preprocessor.transform(X_val_raw, y_val_raw)

# preprocessor.plot_correlation(X_train, y_train)

##


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">توضیحات</h3>
</div>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
در این بخش از مدل های درختی استفاده شده که بر اساس مقایسه و برش زدن داده ها کار می کنند بنابراین نیازی به scale کردن برای عملکرد صحیح آنها نیست هرچند در این کد اضافه شده تا به صورت عمومی برای مدل های خطی هم بتواند مورد استفاده قرار بگیرد
</p>

<p>
برای تبدیل مقادیر متنی به عددی از روش Label Encoding استفاده شده که در آن به هر مقدار یونیک موجود در هر ستون یک عدد نسبت داده می شود علت استفاده این است که در مدل های درختی ایجاد ترتیب مصنوعی در ستون های متنی باعث عملکرد نادرست آن نمی شود زیرا این مدل ها صرفا بر اساس مقایسه کار می کنند و به دنبال نقطه برش بهینه هستند بنابر این نیازی به افزایش ابعاد جدول برای استفاده از روش one hot نبود
</p>

</div>

<table style="width: 100%; text-align: center; margin: 25px 0;">
    <tr>
        <td><img src="./Assets/correlation_matrix_part3.png" width="800"></td>
    </tr>
</table>

<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">تفسیر ماتریس correlation</h3>
</div>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
در مجموع، روابط مشاهده‌شده بین ویژگی‌ها منطقی به نظر می‌رسند. همبستگی بسیار بالای بین دو ویژگی <em>num_assignments_done</em> و <em>assignments_done_ratio</em> نشان می‌دهد که این دو متغیر تا حد زیادی اطلاعات تکراری در اختیار مدل قرار می‌دهند و در صورت نیاز می‌توان یکی از آن‌ها را بدون از دست رفتن اطلاعات معنادار حذف کرد. از طرف دیگر، پایین بودن مقادیر همبستگی پیرسون بین ویژگی‌ها و متغیر هدف لزوما به معنای ضعیف یا بی‌اهمیت بودن ویژگی‌ها نیست. این مسئله بیشتر به غیرخطی بودن روابط بین متغیرها و نحوه کدگذاری برچسب هدف با استفاده از Label Encoding بازمی‌گردد، که باعث می‌شود رابطه واقعی به صورت خطی قابل مشاهده نباشد. در چنین شرایطی، معیارهایی مانند همبستگی خطی توان نمایش تعاملات پیچیده بین ویژگی‌ها را ندارند و به همین دلیل استفاده از مدل‌های غیرخطی مانند Decision Tree و Random Forest برای این مسئله انتخاب مناسبی محسوب می‌شود، زیرا این مدل‌ها قادرند روابط شرطی و غیرخطی بین ویژگی‌ها و متغیر هدف را بهتر استخراج کنند.
</p>



</div>




# Decision Tree Training and Validation Results

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree


dt_model = DecisionTreeClassifier(max_depth=4, random_state=42)
dt_model.fit(X_train, y_train)
y_pred = dt_model.predict(X_val)

acc = accuracy_score(y_val, y_pred)
print(f"Overall Accuracy: {acc:.4f}")

report = classification_report(y_val, y_pred, target_names=preprocessor.target_encoder.classes_, output_dict=True)
print(f"Macro Precision: {report['macro avg']['precision']:.4f}")
print(f"Macro Recall:    {report['macro avg']['recall']:.4f}")
print(f"Macro F1-Score:  {report['macro avg']['f1-score']:.4f}")

cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=preprocessor.target_encoder.classes_,
            yticklabels=preprocessor.target_encoder.classes_)
plt.title("Confusion Matrix - Decision Tree")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

plt.figure(figsize=(20, 10))
plot_tree(dt_model, 
          feature_names=preprocessor.num_cols + preprocessor.cat_cols, 
          class_names=preprocessor.target_encoder.classes_, 
          filled=True, fontsize=10)
plt.title("Decision Tree Visualization")
plt.show()



##

<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">توضیحات</h3>
</div>

<table style="width: 100%; text-align: center; margin: 25px 0;">
    <tr>
        <td><img src="./Assets/3class_res_part3.png " width="800"></td>
        <td><img src="./Assets/3class_tree_part3.png" width="800"></td>
    </tr>
    <tr>
        <td>overall results</td>
        <td>tree</td>
    </tr>
</table>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
۱. تفاوت متریک‌ها: در حالت چندکلاسه از روش Macro-Averaging استفاده شده است. در این روش، ابتدا مسئله به چندین مسئله‌ی باینری (One-vs-Rest) شکسته می‌شود و متریک‌ها برای هر کلاس جداگانه محاسبه می‌گردند. سپس میانگین حسابی (بدون وزن‌دهی) آن‌ها گرفته می‌شود تا عملکرد مدل روی تمام کلاس‌ها با اهمیت یکسان سنجیده شود.

۲. مهم‌ترین ویژگی‌ها: طبق آنتروپی/جینی در درخت تصمیم، ویژگی stress_level بیشترین اهمیت را در جداسازی گره‌ها داشته است و پس از آن num_assignments_done و sleep_hours_before_exam قرار دارند. این نشان می‌دهد وضعیت روانی و خواب دانشجو مهم‌ترین شاخص‌های بحران هستند.

۳. تحلیل خطا: با بررسی ماتریس درهم‌ریختگی (Confusion Matrix)، بیشترین خطای مدل مربوط به کلاس Denial_mode است که اشتباهاً به عنوان No_issue پیش‌بینی می‌شود. دلیل این امر شباهت الگوی رفتاری دانشجویان در حال انکار با دانشجویان بدون مشکل است.
</p>

</div>




# Ensemble Learning

## Bagging

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV


rf_base = RandomForestClassifier(random_state=42)

rf_param_grid = {
    'n_estimators': [50, 100, 200, 300],        # number of trees in the forest
    'max_depth': [None, 10, 20, 30],            # maximum depth of each tree
    'min_samples_split': [2, 5, 10],            # minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],              # minimum samples required at each leaf node
    'bootstrap': [True, False]                  # bootstrap sampling method true means sampling with replacement and false means without replacement
}

rf_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=rf_param_grid,
    n_iter=20,
    cv=3,                 # number of cross-validation splits
    verbose=1,
    random_state=42,
    n_jobs=-1             # use all available cores
)

rf_search.fit(X_train, y_train)
best_rf_model = rf_search.best_estimator_

y_rf_pred = best_rf_model.predict(X_val)

print(f"Accuracy: {accuracy_score(y_val, y_rf_pred):.4f}")

print("\nClassification Report:\n")
print(classification_report(y_val, y_rf_pred))

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_val, y_rf_pred), annot=True, fmt='d', cmap='Greens')
plt.title("Confusion Matrix - Tuned Random Forest")
plt.show()

### result

<table style="width: 100%; text-align: center; margin: 25px 0;">
    <tr>
        <td><img src="./Assets/bagging_model_res.png" width="600"></td>
    </tr>
</table>



## Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV


gb_base = GradientBoostingClassifier(random_state=42)

gb_param_grid = {
    'n_estimators': [50, 100],      # number of trees in the forest   
    'learning_rate': [0.01, 0.1, 0.2],  # learning rate
    'max_depth': [3, 5]               # maximum depth of each tree
}

gb_search = GridSearchCV(
    estimator=gb_base,
    param_grid=gb_param_grid,
    cv=3,
    verbose=1,
    n_jobs=-1
)

gb_search.fit(X_train, y_train)

best_gb_model = gb_search.best_estimator_
y_pred_gb = best_gb_model.predict(X_val)

print(f"Accuracy: {accuracy_score(y_val, y_pred_gb):.4f}")

print("\nClassification Report:\n")
print(classification_report(y_val, y_pred_gb))

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_val, y_pred_gb), annot=True, fmt='d', cmap='Oranges')
plt.title("Confusion Matrix - Tuned Gradient Boosting")
plt.show()

### result

<table style="width: 100%; text-align: center; margin: 25px 0;">
    <tr>
        <td><img src="./Assets/boosting_model_result.png" width="600"></td>
    </tr>
</table>


## سوالات

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
روش One-Hot Encoding به طور کلی برای مدل‌های خطی مناسب‌تر است، زیرا در Label Encoding به مقادیر متنی که ذاتا هیچ‌گونه برتری یا ترتیب معنایی نسبت به یکدیگر ندارند، به صورت ضمنی یک ترتیب عددی نسبت داده می‌شود. این موضوع در مدل‌های درختی معمولا مشکل‌ساز نیست، چرا که این مدل‌ها بر اساس مقایسه و تقسیم داده‌ها عمل می‌کنند و به فاصله عددی بین مقادیر توجهی ندارند. اما در مدل‌های خطی، این مسئله می‌تواند باعث بروز خطا شود، زیرا این مدل‌ها مقادیر عددی را مستقیما در روابط ریاضی خود وارد می‌کنند و در نتیجه به برچسبی که عدد بزرگ‌تری به آن اختصاص داده شده، وزن و اهمیت بیشتری می‌دهند. در حالی که در واقعیت، تمامی مقادیر متنی از اهمیت یکسانی برخوردار هستند و هیچ ترتیب ذاتی بین آن‌ها وجود ندارد.
</p>

</div>




# One-vs-Rest Classification

In [ ]:
class_names = preprocessor.target_encoder.classes_ 
models = {}

for class_idx, class_label in enumerate(class_names):
    y_train_binary = (y_train == class_idx).astype(int)
    
    dt = DecisionTreeClassifier(max_depth=5, random_state=42)
    dt.fit(X_train, y_train_binary)
    
    models[class_idx] = dt

probilities = np.zeros((len(X_val), 3))

for class_idx, class_label in enumerate(class_names):
    probilities[:, class_idx] = models[class_idx].predict_proba(X_val)[:, 1]

y_pred = np.argmax(probilities, axis=1)

acc = accuracy_score(y_val, y_pred)
print(f"Overall Accuracy: {acc:.4f}")

print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=class_names))

plt.figure(figsize=(7, 6))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples', 
            xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix - One vs Rest Strategy")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()



### result

<table style="width: 100%; text-align: center; margin: 25px 0;">
    <tr>
        <td><img src="./Assets/One-vs-Rest_res.png" width="600"></td>
    </tr>
</table>


## سوالات

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
وقتی از Label Encoding استفاده می‌کنیم به هر کلاس یک عدد اختصاص داده می‌شود که این اعداد فقط شناسه آن کلاس ها هستند، اما بسیاری از مدل‌ها آن‌ها را به عنوان مقدار عددی واقعی در نظر می‌گیرند و دچار خطا می شوند اما درخت‌های تصمیم چون بر اساس مقایسه و برش شرطی داده‌ها عمل می‌کنند و روی لیبل‌ها عملیات عددی انجام نمی‌دهند دچار خطا نمی شوند.

تفاوت اصلی این دو روش این در نحوه تعریف مسئله است. در حالت استفاده از Label Encoding در یک مدل چندکلاسه، فقط یک ستون هدف داریم که مقادیر گسسته‌ای مثل 0، 1 و 2 را می‌گیرد. در این حالت، مدل تلاش می‌کند مرزهای تصمیم‌گیری را طوری یاد بگیرد که همه کلاس‌ها را همزمان از هم جدا کند.

اما در روش One-vs-Rest با استفاده از One-Hot Labels، مسئله چندکلاسه به چند مسئله باینری مستقل شکسته می‌شود. برای هر کلاس یک مدل جدا آموزش داده می‌شود که فقط پاسخ یک سوال ساده را بدهد: «آیا این نمونه متعلق به این کلاس هست یا نه؟». در این حالت، تمام کلاس‌های دیگر با هم یکی شده و به عنوان کلاس منفی در نظر گرفته می‌شوند. به این ترتیب هر مدل فقط روی تشخیص کلاس خودش تمرکز می‌کند و کاری به تفاوت‌های جزئی بین سایر کلاس‌ها ندارد. این رویکرد هم مشکل ترتیب‌پذیری کاذب را از بین می‌برد و هم باعث می‌شود مدل‌ها ویژگی‌های شاخص هر کلاس را واضح‌تر یاد بگیرند.
</p>



</div>


# Runner

In [ ]:
import pandas as pd
import numpy as np
import os

test_file = "./test/part3_test.csv"      
output_folder = "part3_submissions"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print(f"Reading test file: {test_file} ...")
df_test = pd.read_csv(test_file)

print("Preprocessing test data...")
X_test_transformed = preprocessor.transform(df_test)

idx_to_label = {0: 'Denial_mode', 1: 'No_issue', 2: 'panic_mode'}

def generate_submission(model, X, original_df, filename, model_name):
    print(f"Generating results for: {model_name}...")
    
    pred_indices = model.predict(X)
    
    pred_labels = [idx_to_label[idx] for idx in pred_indices]
    
    submission = original_df.copy()
    submission['predicted_label'] = pred_labels
    
    save_path = os.path.join(output_folder, filename)
    submission.to_csv(save_path, index=False)
    print(f"Saved: {save_path}")

generate_submission(
    model=dt_model, 
    X=X_test_transformed, 
    original_df=df_test,
    filename="DecisionTree_results.csv",
    model_name="Decision Tree (Baseline)"
)

generate_submission(
    model=best_rf_model, 
    X=X_test_transformed, 
    original_df=df_test,
    filename="RandomForest_results.csv",
    model_name="Random Forest (Tuned)"
)

generate_submission(
    model=best_gb_model, 
    X=X_test_transformed, 
    original_df=df_test,
    filename="GradientBoosting_results.csv",
    model_name="Gradient Boosting (Tuned)"
)

print(f"Generating results for: One-vs-Rest (OvR)...")

probs_matrix = np.zeros((len(X_test_transformed), 3))
probs_matrix[:, 0] = models[0].predict_proba(X_test_transformed)[:, 1] # Denial
probs_matrix[:, 1] = models[1].predict_proba(X_test_transformed)[:, 1] # No_issue
probs_matrix[:, 2] = models[2].predict_proba(X_test_transformed)[:, 1] # Panic

ovr_indices = np.argmax(probs_matrix, axis=1)
ovr_labels = [idx_to_label[idx] for idx in ovr_indices]

submission_ovr = df_test.copy()
submission_ovr['predicted_label'] = ovr_labels
ovr_save_path = os.path.join(output_folder, "OneVsRest_results.csv")
submission_ovr.to_csv(ovr_save_path, index=False)
print(f"Saved: {ovr_save_path}")

print("\n" + "="*50)
print("All submissions generated successfully!")
print("="*50)